In [1]:
from google.colab import drive
drive.mount("/content/drive")

!find "/content/drive" -maxdepth 6 -name "adapter_config.json" -print


Mounted at /content/drive


In [ ]:
!find "/content/drive" -maxdepth 8 -name "adapter_config.json" -print


/content/drive/.shortcut-targets-by-id/14ok9jUkxn5Ww4SgxFGRMgEXs78uQROae/steam_project_lora/steam_lora_adapter/adapter_config.json
find: ‘/content/drive/.Encrypted/.shortcut-targets-by-id/14ok9jUkxn5Ww4SgxFGRMgEXs78uQROae/steam_project_lora’: No such file or directory


In [ ]:
!ls -la "/content/drive/MyDrive"
!ls -la "/content/drive/MyDrive" | head


total 826731
-rw-------  1 root root  5844875 Jul  7 14:35  20250618_154133.jpg
drwx------  2 root root     4096 Dec 15  2022 'anglais proba'
-rw-------  1 root root      175 Jun  5  2025 'appel guillaume gaudron.gdoc'
drwx------  2 root root     4096 Nov 22  2021  bleu
drwx------  3 root root     4096 Jul  4  2024  Borderlands-Sensitivity-Changer-main
-rw-------  1 root root      175 Nov 11 11:05  Bouquins.gdoc
-rw-------  1 root root      175 Mar  2  2020 'Ces idées qui collent.gslides'
-rw-------  1 root root      175 Mar 12  2021 'chaine mentale tipe.gdoc'
-rw-------  1 root root      175 Feb 14  2024 'Chapitre_9-1 (1).gdoc'
-rw-------  1 root root      175 Feb 14  2024  Chapitre_9-1.gdoc
-rw-------  1 root root      175 Feb 18  2024 'Chapitre_9 (1).gdoc'
-rw-------  1 root root      175 Feb 18  2024 'Chapitre_9 (2).gdoc'
-rw-------  1 root root      175 Feb 14  2024 'Chapitre_9 (3).gdoc'
-rw-------  1 root root      175 Feb 14  2024 'Chapitre_9 (4).gdoc'
-rw-------  1 root root  

In [2]:
ADAPTER_DIR = "/content/drive/.shortcut-targets-by-id/14ok9jUkxn5Ww4SgxFGRMgEXs78uQROae/steam_project_lora/steam_lora_adapter"


In [ ]:
import os
print(os.path.isfile(os.path.join(ADAPTER_DIR, "adapter_config.json")))
print(os.path.isfile(os.path.join(ADAPTER_DIR, "adapter_model.safetensors")))


True
True


In [ ]:
# =========================
# STEAM LoRA BATCH GENERATION (ROBUST + RESUME + SAVE TO DRIVE)
# - Reads:  /content/prompt_batch.csv (already uploaded)
# - Loads adapter from: Drive shortcut targets (path below)
# - Writes output directly to Drive (so you won't lose progress if Colab resets)
# - Resumes automatically if the output file already exists
# =========================

!pip -q install transformers accelerate bitsandbytes peft pandas

import os
import re
import csv
import pandas as pd
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from google.colab import drive

# ---- 0) Mount Drive (required) ----
drive.mount("/content/drive")

# ---- 1) Paths ----
INPUT_CSV = "/content/prompt_batch.csv"

# Adapter path (from your find result)
ADAPTER_DIR = "/content/drive/.shortcut-targets-by-id/14ok9jUkxn5Ww4SgxFGRMgEXs78uQROae/steam_project_lora/steam_lora_adapter"

# Write directly to Drive (persistent)
OUTPUT_CSV = "/content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv"

# ---- 2) Generation settings (T4 safe) ----
BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
MAX_NEW_TOKENS = 220
BATCH_SIZE = 4                 # if OOM -> set 2
SAVE_EVERY_N_BATCHES = 5       # flush every 5 batches (~20 rows)

# ---- 3) Sanity checks ----
assert os.path.isfile(INPUT_CSV), f"Missing INPUT_CSV: {INPUT_CSV}"
assert os.path.isfile(os.path.join(ADAPTER_DIR, "adapter_config.json")), "adapter_config.json not found in ADAPTER_DIR"
assert (
    os.path.isfile(os.path.join(ADAPTER_DIR, "adapter_model.safetensors"))
    or os.path.isfile(os.path.join(ADAPTER_DIR, "adapter_model.bin"))
), "adapter model weights not found (adapter_model.safetensors/bin)"

print("INPUT_CSV OK:", INPUT_CSV)
print("ADAPTER_DIR OK:", ADAPTER_DIR)
print("OUTPUT_CSV:", OUTPUT_CSV)

# Optional (recommended): copy adapter locally for speed/stability
LOCAL_ADAPTER = "/content/steam_lora_adapter"
if not os.path.isdir(LOCAL_ADAPTER):
    !rm -rf /content/steam_lora_adapter
    !cp -r "$ADAPTER_DIR" /content/steam_lora_adapter
ADAPTER_DIR = LOCAL_ADAPTER
print("Using local adapter dir:", ADAPTER_DIR)

# ---- 4) Load tokenizer/model + attach adapter ----
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

# Tokenizer from base model (safe)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
)

model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()

def clean_generated(text: str) -> str:
    t = str(text).strip()
    t = re.sub(r"^\s*review\s*:\s*", "", t, flags=re.IGNORECASE)
    return t.strip()

@torch.inference_mode()
def generate_batch(prompts: list[str]) -> list[str]:
    enc = tokenizer(
        prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
    enc = {k: v.to(model.device) for k, v in enc.items()}
    input_lens = enc["attention_mask"].sum(dim=1)

    out = model.generate(
        **enc,
        min_new_tokens=130,
        max_new_tokens=190,

        do_sample=True,
        temperature=0.75,
        top_p=0.9,
        repetition_penalty=1.15,
        no_repeat_ngram_size=3,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True,
    )

    results = []
    for i in range(out.size(0)):
        gen_ids = out[i, input_lens[i]:]
        txt = tokenizer.decode(gen_ids, skip_special_tokens=True)
        results.append(clean_generated(txt))
    return results

# ---- 5) Load prompts + keep engineered ----
df = pd.read_csv(INPUT_CSV, engine="python")
df["method"] = df["method"].astype(str)
df["generated_text"] = df["generated_text"].fillna("").astype(str)

to_gen = df[df["method"] == "engineered"].copy().reset_index(drop=True)
print("Engineered rows to generate:", len(to_gen))

# ---- 6) Resume logic (if OUTPUT_CSV already exists on Drive) ----
start_idx = 0
if os.path.exists(OUTPUT_CSV):
    done = pd.read_csv(OUTPUT_CSV, engine="python")
    start_idx = len(done)
    print(f"Resuming: {start_idx} already generated, remaining: {len(to_gen) - start_idx}")
else:
    # Create empty output with header
    header_df = to_gen.iloc[0:0].copy()
    header_df["generated_text"] = ""
    header_df.to_csv(
        OUTPUT_CSV,
        index=False,
        encoding="utf-8",
        quoting=csv.QUOTE_ALL,
        lineterminator="\n",
    )
    print("Created output file:", OUTPUT_CSV)

# ---- 7) Streaming generation + periodic flush to Drive ----
rows_buffer = []
batch_counter = 0

for i in range(start_idx, len(to_gen), BATCH_SIZE):
    batch = to_gen.iloc[i:i+BATCH_SIZE].copy()
    prompts = batch["prompt"].tolist()

    gens = generate_batch(prompts)

    batch["generated_text"] = gens
    batch["method"] = "finetuned_v5"
    rows_buffer.append(batch)
    batch_counter += 1

    # Flush every N batches
    if batch_counter % SAVE_EVERY_N_BATCHES == 0:
        chunk = pd.concat(rows_buffer, ignore_index=True)
        chunk.to_csv(
            OUTPUT_CSV,
            mode="a",
            header=False,
            index=False,
            encoding="utf-8",
            quoting=csv.QUOTE_ALL,
            lineterminator="\n",
        )
        rows_buffer = []
        print(f"Saved progress: {min(i + BATCH_SIZE, len(to_gen))}/{len(to_gen)} rows -> {OUTPUT_CSV}")

# Flush remaining
if rows_buffer:
    chunk = pd.concat(rows_buffer, ignore_index=True)
    chunk.to_csv(
        OUTPUT_CSV,
        mode="a",
        header=False,
        index=False,
        encoding="utf-8",
        quoting=csv.QUOTE_ALL,
        lineterminator="\n",
    )

print("DONE. Output saved on Drive:", OUTPUT_CSV)

# ---- 8) Quick quality check ----
out = pd.read_csv(OUTPUT_CSV, engine="python")
wc = out["generated_text"].fillna("").str.split().str.len()
print("Output rows:", len(out))
print("Mean words:", float(wc.mean()))
print("Share 100-140:", float(((wc >= 100) & (wc <= 140)).mean()))
print("\nLast sample (first 350 chars):\n", out["generated_text"].iloc[-1][:350])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.9 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INPUT_CSV OK: /content/prompt_batch.csv
ADAPTER_DIR OK: /content/drive/.shortcut-targets-by-id/14ok9jUkxn5Ww4SgxFGRMgEXs78uQROae/steam_project_lora/steam_lora_adapter
OUTPUT_CSV: /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Using local adapter dir: /content/steam_lora_adapter


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Saved progress: 940/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 960/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 980/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1000/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1020/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1040/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1060/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1080/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1100/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Save

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Saved progress: 1660/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1680/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1700/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1720/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1740/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1760/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1780/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1800/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 1820/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
S

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Saved progress: 3460/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3480/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3500/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3520/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3540/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3560/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3580/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3600/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
Saved progress: 3620/4422 rows -> /content/drive/MyDrive/steam_project_lora/prompt_batch_finetuned_v6_long.csv
S